# Probe Data - Map Matching
## Nick Paras | Kapil Garg

### Assignment 2

Input: Probe data and map [probe_data_map_matching.rar](https://canvas.northwestern.edu/courses/51440/files/3334329/download?wrap=1)

-The raw probe points in Germany collected in 9 months

-The link data for the links that probe points can be map-matched to.

Tasks:
-- map match probe points to road links

-- derive road slope for each road link

-- evaluate the derived road slope with the surveyed road slope in the link data file

**Please submit your code and slides presentation of your approach and results including evaluation comparing with the slopes in the link data file**

### Setup

We use **Python 3.6** and rely on a number of dependencies for computation and visualization. To easily install everything, we have included all of our dependencies in `environment.yml`. For quick setup, please create a conda environment with the following:

    $ conda create --name probe-data -f environment.yml

and then activate the conda environment with

    $ source activate probe-data

In [4]:
# Imports
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import gmplot

from datetime import datetime
from bs4 import BeautifulSoup
from IPython.display import IFrame

%matplotlib inline

# Constants
DATA_DIR = "probe_data_map_matching"

GOOGLE_MAPS_KEY = ''
with open('config.json') as data_file:
    data = json.load(data_file)
    GOOGLE_MAPS_KEY = data['google-maps-key']

## Loading Probe Data for Map Matching

Here we'll load our data from the two csv's into Pandas DataFrames.

In [5]:
probe_headers = ['sampleID', 
                 'dateTime', 
                 'sourceCode', 
                 'latitude', 
                 'longitude', 
                 'altitude', 
                 'speed', 
                 'heading']

probe_data = pd.read_csv(os.path.join(DATA_DIR, 'Partition6467ProbePoints.csv'), header=None, names=probe_headers)
probe_data.drop_duplicates(inplace=True)
probe_data['dateTime'] = pd.to_datetime(probe_data['dateTime'], format='%m/%d/%Y %I:%M:%S %p')
probe_data.head()

,sampleID,dateTime,sourceCode,latitude,longitude,altitude,speed,heading
0,3496,2009-06-12 06:12:49,13,51.496868,9.386022,200,23,339
1,3496,2009-06-12 06:12:54,13,51.496682,9.386157,200,10,129
2,3496,2009-06-12 06:12:59,13,51.496705,9.386422,201,21,60
3,3496,2009-06-12 06:13:04,13,51.496749,9.386840,201,0,360
4,3496,2009-06-12 06:13:09,13,51.496864,9.387294,199,0,360


In [6]:
link_headers = ['linkPVID', 
                'refNodeID', 
                'nrefNodeID', 
                'length', 
                'functionalClass', 
                'directionOfTravel', 
                'speedCategory', 
                'fromRefSpeedLimit', 
                'toRefSpeedLimit', 
                'fromRefNumLanes', 
                'toRefNumLanes', 
                'multiDigitized', 
                'urban', 
                'timeZone', 
                'shapeInfo', 
                'curvatureInfo', 
                'slopeInfo']

# load raw link data
link_data = pd.read_csv(os.path.join(DATA_DIR, 'Partition6467LinkData.csv'), header=None, names=link_headers)
link_data['speedLimit'] = link_data[['fromRefSpeedLimit', 'toRefSpeedLimit']].max(axis=1)
link_data.loc[link_data['speedLimit'] == 0, 'speedLimit'] = 1
link_data['shapeArray'] = link_data['shapeInfo'].apply(lambda x: [[float(j) for j in i.split('/')[:2]] for i in x.split('|')])
link_data['location'] = link_data['shapeArray'].apply(lambda x: reduce(np.add, x) / len(x))
link_data.head()

NameError: name 'reduce' is not defined

#### Initial Observations

As can be seen in the first 4 rows of `link_data`, there are missing values in the data. More specifically, we can see right away that `curvatureInfo` and `slopeInfo` are missing from some rows. After checking the `README`, we confirm that this is expected.

### Exploratory Data Analysis

#### Number of Points per Trajectory, Sample Rate, etc.

We know from the literature that the sample rate and/or number of points per trajectory can have a big impact on the quality of our matching, and should therefore inform our choice of algorithm.

In [ ]:
sample_sizes = probe_data.groupby(['sampleID'])['dateTime'].size()

In [ ]:
len(sample_sizes)

So, there are 75840 unique trajectories.

In [ ]:
sample_sizes.hist()

And, it appears that most of them have fewer than 100 sample points, although there are some that have quite a bit more.

Now we want to find the sampling rate of the points.

In [ ]:
sampling_rates = probe_data.groupby(['sampleID'])

In [ ]:
date_bounds = sampling_rates['dateTime'].agg({'date_min': min,
                                              'date_max': max})

num_samples = sampling_rates['dateTime'].size()

In [ ]:
date_bounds.head()

In [ ]:
num_samples.head()

In [ ]:
combined = date_bounds
combined['num_samples'] = num_samples

In [ ]:
combined.head()

In [ ]:
combined['time_diff'] = combined['date_max'] - combined['date_min']

In [ ]:
combined['time_diff'] = combined['time_diff'].apply(lambda x: x.seconds / 60.0)

In [ ]:
combined['sample_rate'] = combined['num_samples']/(combined['time_diff'] + 0.00001)

In [ ]:
combined['sample_rate'].hist(range=(0,20))

So, we can see that most of the trajectories are sampled between 3 and 15 Hz.

## Simple Point-to-Curve Matching

First, we start with the simplest possible method, _point-to-curve_. We know from the literature that this method (and other simple techniques) is/are highly sensitive to outliers. Therefore, we start with this method more to establish our data pipeline rather than out of expectation of a final solution.

In [ ]:
probe_data.head()

In [ ]:
probe_data.tail()

In [ ]:
trajectories = probe_data.groupby(['sampleID'])

In [ ]:
trajectories.get_group(3496).head()

In [ ]:
link_data.head()

In [ ]:
link_data.loc[345]

In [ ]:
from haversine import haversine

In [ ]:
haversine((51.4256599, 10.0942899), (51.4257300, 10.0941699))

In [ ]:
# split the lat/lons out from the string
[[float(x) for x in i.split('/')[:2]] for i in link_data.loc[345]['shapeInfo'].split('|')]

In [ ]:
link_data[['linkPVID', 'shapeInfo']].head()

In [ ]:
def find_closest(probe_point):
    tmp_col = link_data[['linkPVID', 'shapeInfo']]
    tmp_col['min_link_dist'] = tmp_col['shapeInfo'].apply(lambda x: min(haversine(probe_point, [float(j) for j in i.split('/')[:2]]) for i in x.split('|')))
    return tmp_col.ix[tmp_col['min_link_dist'].idxmin()]['linkPVID']

In [ ]:
find_closest([51.496868, 9.386022])

In [ ]:
find_closest([52.217058, 8.974134])

In [ ]:
probe_data[['sampleID', 'latitude', 'longitude']].head().apply(lambda x: find_closest((x[1], x[2])), axis=1)

In [ ]:
probe_data['pointToCurveNearest'] = probe_data[['sampleID', 'latitude', 'longitude']].apply(lambda x: find_closest((x[1], x[2])), axis=1)

### Plot data
Now, we plot both the probe data with its associated links

In [7]:
simple_match_data = pd.read_csv('./simple_match.csv')
simple_match_data.head()

,sampleID,dateTime,sourceCode,latitude,longitude,altitude,speed,heading,linkPVID,direction,distFromRef,distFromLink
0,3496,6/12/2009 6:12:49 AM,13,51.496868,9.386022,200,23,339,706849300.0,B,360.857004,2.264648
1,3496,6/12/2009 6:12:54 AM,13,51.496682,9.386157,200,10,129,62007637.0,B,12.429214,1.654407
2,3496,6/12/2009 6:12:59 AM,13,51.496705,9.386422,201,21,60,550095206.0,B,560.570762,0.653992
3,3496,6/12/2009 6:13:04 AM,13,51.496749,9.386840,201,0,360,567329767.0,B,46.237135,2.528689
4,3496,6/12/2009 6:13:09 AM,13,51.496864,9.387294,199,0,360,567329767.0,B,80.145068,3.234761


In [8]:
def make_map_plot(method, sample_id, gmaps_api_key, data):
    probe_plot_data = data[(data['linkPVID'] != 0) & (data['sampleID'] == sample_id)]

    # create map object centered at mean lat, long
    gmap = gmplot.GoogleMapPlotter(np.mean(probe_plot_data['latitude']), np.mean(probe_plot_data['longitude']), 16)

    # plot data with color-coded probes and links
    unique_links = probe_plot_data['linkPVID'].unique()
    colors = list(gmap.color_dict.keys())[0:-1]
    color_index = 0

    for i in unique_links:
        # setup variables
        current_color = colors[color_index]
        probe_lats = probe_plot_data[probe_plot_data['linkPVID'] == i]['latitude']
        probe_longs = probe_plot_data[probe_plot_data['linkPVID'] == i]['longitude']

        link_lats = [x[0] for x in list(link_data[link_data['linkPVID'] == i]['shapeArray'])[0]]
        link_longs = [x[1] for x in list(link_data[link_data['linkPVID'] == i]['shapeArray'])[0]]
        
        gmap.scatter(probe_lats, probe_longs, marker=False, color=current_color, s=5)
        gmap.plot(link_lats, link_longs, color=current_color, edge_width=10, alpha=0.25)

        color_index = (color_index + 1) % len(colors)
        print('Link Segment: ' + str(i) + ', Color: ' + str(current_color))
    
    # print out file
    if not os.path.exists('./graphs'):
        os.makedirs('./graphs')
    file_name = './graphs/' + method + '_' + str(sample_id) + '.html'
    gmap.draw(file_name)

    def insertapikey(fname, apikey):
        """put the google api key in a html file"""
        def putkey(htmltxt, apikey, apistring=None):
            """put the apikey in the htmltxt and return soup"""
            if not apistring:
                apistring = 'https://maps.googleapis.com/maps/api/js?key=%s&callback=initMap'
            soup = BeautifulSoup(htmltxt, 'html.parser')
            body = soup.body
            src = apistring % (apikey, )
            tscript = soup.new_tag('script', src=src, async='defer')
            body.insert(-1, tscript)
            return soup
        htmltxt = open(fname, 'r').read()
        soup = putkey(htmltxt, apikey)
        newtxt = soup.prettify()
        open(fname, 'w').write(newtxt)

    insertapikey(file_name, gmaps_api_key)
    return IFrame(file_name, width=985, height=700)

In [17]:
# select data to plot
sample_id = 3496
make_map_plot('simple-match', sample_id, GOOGLE_MAPS_KEY, simple_match_data)

Link Segment: 706849300.0, Color: b
Link Segment: 62007637.0, Color: g
Link Segment: 550095206.0, Color: r
Link Segment: 567329767.0, Color: c
Link Segment: 62007648.0, Color: m
Link Segment: 62005171.0, Color: y
Link Segment: 78670326.0, Color: k
Link Segment: 78654476.0, Color: b
Link Segment: 62006493.0, Color: g


In [18]:
# select data to plot
sample_id = 5840302
make_map_plot('simple-match', sample_id, GOOGLE_MAPS_KEY, simple_match_data)

Link Segment: 79685530.0, Color: b
Link Segment: 79685644.0, Color: g
Link Segment: 540652112.0, Color: r
Link Segment: 540652571.0, Color: c
Link Segment: 540652103.0, Color: m
Link Segment: 540652102.0, Color: y
Link Segment: 79926342.0, Color: k
Link Segment: 51796317.0, Color: b
Link Segment: 586504920.0, Color: g
Link Segment: 540652572.0, Color: r
Link Segment: 51796322.0, Color: c
Link Segment: 79687459.0, Color: m
Link Segment: 540650979.0, Color: y
Link Segment: 79687447.0, Color: k
Link Segment: 79772029.0, Color: b


In [16]:
# select data to plot
sample_id = 778178
make_map_plot('simple-match', sample_id, GOOGLE_MAPS_KEY, simple_match_data)

Link Segment: 67942589.0, Color: b
Link Segment: 67942583.0, Color: g
Link Segment: 67942585.0, Color: r
Link Segment: 586503844.0, Color: c
Link Segment: 586484912.0, Color: m
Link Segment: 554724701.0, Color: y
Link Segment: 554721747.0, Color: k
Link Segment: 554724814.0, Color: b
Link Segment: 572216129.0, Color: g
Link Segment: 572196708.0, Color: r
Link Segment: 51901135.0, Color: c
Link Segment: 51901184.0, Color: m
Link Segment: 781679461.0, Color: y
Link Segment: 781670235.0, Color: k
Link Segment: 781670234.0, Color: b
Link Segment: 781670233.0, Color: g
Link Segment: 51901557.0, Color: r
Link Segment: 51901569.0, Color: c
Link Segment: 51931832.0, Color: m
Link Segment: 572216467.0, Color: y
Link Segment: 51931828.0, Color: k
Link Segment: 586503845.0, Color: b
Link Segment: 51900862.0, Color: g
Link Segment: 51901444.0, Color: r
Link Segment: 51901342.0, Color: c
Link Segment: 572216117.0, Color: m
Link Segment: 572216132.0, Color: y
Link Segment: 586503847.0, Color: k
Link